# 1. ETAPA: One Vs Rest

## Entrenamiento con las proteínas con activaciones mayores a 100

Uno de los objetivos en este inicio de la primera etapa es reducir el problema de 206 proteínas a solo 41 (siendo estas las de mayor activación o activaciones mayores a 100. Esto significa que más de 100 fármacos generaron que se activaran). Esto para reducir el problema y tener una solución inicial.

Para las librerías que deben ser usadas, es importante que se instale el módulo `pip install scikit-multilearn`, ya que este no permite hacer una partición iterativa de los datos para entrenamiento y prueba, cuando se tiene problemas desbalanceados. Otros de los módulo a instalar es `pip install xgboost`, ya que este modelo de boosting es uno de los usados en la primera etapa de la experimentación.

## Importación de librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import scipy.stats as stats
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedShuffleSplit, ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score, plot_confusion_matrix, confusion_matrix, log_loss
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.svm import SVC
from sklearn.metrics._plot.confusion_matrix import ConfusionMatrixDisplay
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import seaborn as sn
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

In [ ]:
train_features = pd.read_csv('train_features.csv')
train_target = pd.read_csv('train_targets_scored.csv')
test_features = pd.read_csv('test_features.csv')

## Preparación del dataset

In [ ]:
train_features['cp_time'] = train_features['cp_time'].map({24:1, 48:2, 72:3})
train_features['cp_dose'] = train_features['cp_dose'].map({'D1':0, 'D2':1})

In [ ]:
data_train = pd.concat([train_features, train_target], axis = 1)
train = data_train[data_train['cp_type'] == 'trt_cp']
evaluar = test_features[test_features['cp_type'] == 'trt_cp']

In [ ]:
# Selección de datos
X = train.iloc[:,2:876]
y = train.iloc[:,877:]
x_eval = evaluar.iloc[:,4:]

## Conteo de la cantidad de activaciones

In [ ]:
higher = [col for col in y.columns if (y[col].sum() > 100)]
ytarget = y[higher]

In [ ]:
higher = [col for col in y.columns if (y[col].sum() > 300)]
ytarget = y[higher]

In [ ]:
high_10 = ytarget.sum(axis=0)

In [ ]:
fig = plt.figure(figsize=(12, 15))
sns.barplot(x=ytarget.sum(axis=0).sort_values(ascending=False).values,
            y=ytarget.sum(axis=0).sort_values(ascending=False).index)
plt.tick_params(axis='x', labelsize=12)
plt.tick_params(axis='y', labelsize=12)
# plt.xticks(rotation=45)
plt.xlabel('')
plt.ylabel('')
plt.title('Conteo proteínas con activaciones mayor a 100', size=15, pad=15)

plt.show()

## Conteo por tipo de MoA (proteína)

### 1) Inhibidores

In [ ]:
fig = plt.figure(figsize=(12, 8))
inhibitor = high_10.loc[high_10.index.str.contains('_inhibitor')]
sns.barplot(x=inhibitor.sort_values(ascending=False).values,
            y=inhibitor.sort_values(ascending=False).index)
plt.tick_params(axis='x', labelsize=12)
plt.tick_params(axis='y', labelsize=12)
# plt.xticks(rotation=30)
plt.xlabel('')
plt.ylabel('')
plt.title('Mayor a 100 (Inhibidores)', size=15, pad=15)
plt.show()

### 2) Antagonistas

In [ ]:
fig = plt.figure(figsize=(10, 5))
antago = high_10.loc[high_10.index.str.contains('_antagonist')]
sns.barplot(x=antago.sort_values(ascending=False).values,
            y=antago.sort_values(ascending=False).index)
plt.tick_params(axis='x', labelsize=12)
plt.tick_params(axis='y', labelsize=12)
# plt.xticks(rotation=90)
plt.xlabel('')
plt.ylabel('')
plt.title('Mayor a 100 (Antagonistas)', size=15, pad=15)
plt.show()

### 3) Agonista

In [ ]:
fig = plt.figure(figsize=(10, 5))
agonist = high_10.loc[high_10.index.str.contains('_agonist')]
sns.barplot(x=agonist.sort_values(ascending=False).values,
            y=agonist.sort_values(ascending=False).index)
plt.tick_params(axis='x', labelsize=12)
plt.tick_params(axis='y', labelsize=12)
# plt.xticks(rotation=90)
plt.xlabel('')
plt.ylabel('')
plt.title('Mayor a 100 (Agonista)', size=15, pad=15)
plt.show()

In [ ]:
len(np.unique(train['sig_id'])), train.shape

## Multilabel data stratification

In [ ]:
## Probando 4 labels
ytarget_1 = ytarget.iloc[:,0:4]
# ytarget_1.head(5)

In [ ]:
X_train, y_train, X_test, y_test = iterative_train_test_split(np.array(X), np.array(ytarget), test_size = 0.3)

### Verificación usando el split iterativo

In [ ]:
for i, col in enumerate(ytarget.columns):
    # Ptrain = (np.count_nonzero(y_train[:,i:i+1])/len(y_train[:,i:i+1]))*1.02
    Ptrain = np.count_nonzero(y_train[:,i:i+1])*1.2/len(y_train[:,i:i+1])
    print('Proteina =', col)
    print(round(Ptrain, 4))

In [ ]:
for i, col in enumerate(ytarget.columns):
    Ptrain = (np.count_nonzero(y_train[:,i:i+1])/len(y_train[:,i:i+1]))*100
    Ptest = (np.count_nonzero(y_test[:,i:i+1])/len(y_test[:,i:i+1]))*100
    print('Proteina =', col)
    print(round(Ptrain, 3), round(Ptest,3))

In [ ]:
for i, col in enumerate(ytarget.columns):
    Ptrain = (np.count_nonzero(y_train[:,i:i+1]))
    Ptest = (np.count_nonzero(y_test[:,i:i+1]))
    print('Proteina =', col)
    print(round(Ptrain, 3), round(Ptest,3))

In [ ]:
nfolds = 5
cv = StratifiedShuffleSplit(n_splits = nfolds)
# cv = StratifiedKFold(n_splits = nfolds)

# Modelos iterative_split

Usando la partición iterativa, son separados los diferentes conjuntos para entrenamiento y prueba. A continuación se encuentran las funciones usadas para la ejecución de cada uno de los modelos

In [ ]:
def loga_loss(y_test, y_pred, eps=1e-15):
    """función para el cálculo de la perdida logarítmica establecida en el concurso de kaggle
    
    y_test --> Variable de salida de prueba
    
    y_pred --> Variable predicha (Predicción de probabilidad de activación)"""
    
    los = np.zeros(y_test.shape)
    n, m = y_test.shape
    y_true = np.clip(y_test, eps, 1-eps)
    for M in range(m):
        for N in range(n):
            log_los = -((y_true[N,M]*np.log(y_pred[N,M]+eps))+((1-y_true[N,M])*np.log(1-y_pred[N,M]+eps)))
            los[N,M] = log_los
    return los

In [ ]:
from sklearn.metrics import make_scorer

scorer = make_scorer(loga_loss, greater_is_better=False)

In [ ]:
def ML_mol(x_train, y_train, parameters, modelo, ML_model=None):
    ''' Función para la evaluación de modelos mediante la aplicación de GridSerachCV que permite la combinación de hiperparámetros y cross-validation:

    x_train --> df de entrenamiento para todas las proteínas creados de manera iterativa y estratificada

    y_train --> Variable de salida de entrenamiento para cada proteína

    parameters --> Hiperparámetros de cada modelo de clasificación

    ML_model --> Modelos de clasificación

    modelo --> Lista que debe ser retornada, almacenando los modelos para cada proteína y que será usada para la construcción de las matrices de confusión'''

    for i, col in enumerate(ytarget.columns):
        #Escalado
        scaler = MinMaxScaler()
#         scaler = RobustScaler()
        pipe = Pipeline(steps=[('scaler', scaler), ('ML_model', ML_model)])

        ##Entrenamiento del modelo con combinaciones de hiperparámetros
        print("PROTEINA =", col)
        gridt = GridSearchCV(estimator=pipe, param_grid=parameters, cv=cv, return_train_score=True, verbose=5, n_jobs=-1, scoring='neg_log_loss')
        gridt.fit(x_train, np.ravel(y_train[:,i:i+1]))

        ## Mejor puntuación (score) para las diferentes cross-validation realizadas
        score = gridt.best_score_ 
        print("Score = ", round(score, 3))

        ## Almacenamiento de los modelos para cada proteína
        modelo.append(gridt)

In [ ]:
def matrix_conf(modelo, x_test, y_test, prediction, c_mat):
    ''' Función para la construcción de las funciones de las matrices de confusión para cada uno de las etiquetas (proteínas) que están siendo evaluadas:

    modelo -> Una lista donde se encuentran guardados los modelos entrenados de cada proteína del paso anterior

    x_test -> df creada para pruebas (de forma iterativa para que funcione para todas las proteínas de forma estratificada)

    y_test -> variable de salida por proteína de forma debidamente estritifica

    prediction -> Lista que debe retornarse con cada una de las predicciones 
    
    c_mat -> Retorna las matrices de confusión para cada proteínas para evaluar una métrica global'''

    for i, col in enumerate(ytarget.columns):

        ##Predicciones
        pred_test = modelo[i].predict(x_test)
        pred_proba = modelo[i].predict_proba(x_test)
        ## Métricas usadas para medir la eficiencia del modelo (para cada proteína)
        accuracy = accuracy_score(y_test[:,i:i+1],pred_test)
        bal_accuracy = balanced_accuracy_score(y_test[:,i:i+1],pred_test)
        recall = recall_score(y_test[:,i:i+1],pred_test)
        precision = precision_score(y_test[:,i:i+1],pred_test)
        log_loss = loga_loss(y_test[:,i:i+1], pred_proba[:,1].reshape(-1,1))
        print("PROTEINA =", col)
        print("Accuracy =", round(accuracy, 3))
        print("Balanced Accuracy =", round(bal_accuracy, 3))
        print("Recall score =", round(recall, 3))
        print("Precision score =", round(precision, 3))
        f1 = f1_score(y_test[:,i:i+1], pred_test)
        print("F1 =", round(f1,3))
        print("Log_loss =", round(np.mean(log_loss),3))

        ##Matriz de confusión
        con_max = confusion_matrix(y_test[:,i:i+1], pred_test)
        disp = plot_confusion_matrix(modelo[i], x_test, y_test[:,i:i+1], display_labels=np.unique(np.unique(y_test[:,i:i+1])), cmap=plt.cm.Blues, normalize=None) #normalize='true')
        disp.ax_.set_title(col)
        plt.show()

        ## Listas creadas para almacenar las predicciones y matrices de confusión
        prediction.append(pred_test)
        c_mat.append(con_max)
        print("-----------------------------------------------")

In [ ]:
def global_metric(ytest, prediction, con_mat, metric=None):
    '''Función para la medición de una métrica global sopesada para cada uno de los modelo evaluados sobre las proteínas:

    ytest --> Variable de salida para cada una de las proteínas

    predictions --> predicciones calculadas para cada de las proteínas

    con_mat --> Matrices de confusión de donde serán extraídos los valores obtenidos para cada uno de los modelos de clasificación'''
    lista = []
    for i in range(y_test.shape[1]):
    
        ##Cálculo de las métricas
        g_m = metric(y_test[:,i:i+1],np.array(prediction[i]))

        ##Valor predichos correctamente
        w_cla = con_mat[i][1,1]
        b_cla = con_mat[i][1,0]

        ##Métrica global
        g_met = (w_cla/(b_cla+w_cla)) * g_m
        lista.append(g_met)

    g_metric = round((np.sum(lista)/len(lista)),3)
    print(f'El valor global de {metric.__name__} es {g_metric}')

In [ ]:
def global_logloss(xtest, ytest, conmat, modelo):
    """Función para calcular la perdida logarítmica (cross-entropy) para cada uno de los modelos. Es importante que esta perdida se mide en función de la probabilidad de que una proteína se active a la aplicación de un fármaco:

    xtest --> dataset de prueba con cada una de las variables (génicas y celulares)

    y_test --> dataset de prueba para cada una de las proteínas

    conmat --> Lista con las matrices de confusión para cada una de las proteínas (es usada para tener una métrica sopesada en función de las activaciones)
    
    modelo --> Lista con los modelos para cada una de las proteínas"""
    
    lista = []
    for i in range(y_test.shape[1]):

        pred_proba = modelo[i].predict_proba(xtest)
        ##Cálculo de las métricas
        g_m = loga_loss(ytest[:,i:i+1], pred_proba[:,1].reshape(-1,1))
#         g_m = log_loss(ytest[:,i:i+1], np.ravel(pred_proba[:,1]))

        ##Valor predichos correctamente
        # w_cla = conmat[i][1,1]
        # b_cla = conmat[i][1,0]

        ##Métrica global
        # g_met = (w_cla/(b_cla+w_cla)) * np.mean(g_m)
#         g_met = (w_cla/(b_cla+w_cla)) * g_m
        lista.append(np.mean(g_m))

    g_metric = round((np.sum(lista)/len(lista)),4)
    # print(f'El valor global la perdida logarítmica para el {modelo.__name__} es {g_metric}')
    print(f'Global log loss --> {g_metric}')

In [ ]:
def global_multiLabel_confusion_matrix(y_test_g,y_est_g):
    n_samples, n_class = y_test_g.shape
    CM = np.zeros((n_class,n_class))
    Temp = np.zeros((1,n_class))
    def acum_CM(y_test,y_est,CM,Temp):
        ind_real = np.asarray(y_test > 0).nonzero()[0]
        ind_est = np.asarray(y_est > 0).nonzero()[0]
        #--------------------------------
        if ind_real.size == 0:
            #In case in the ground truth not even one class is active
            Temp = Temp + y_est
        elif ind_est.size == 0:
            return CM, Temp
        else:
            mesh_real = np.array(np.meshgrid(ind_real,ind_real))
            comb_real = mesh_real.T.reshape(-1, 2)
            ind_remove_real = comb_real[:,0] != comb_real[:,1]
            comb_real = comb_real[ind_remove_real]
            #--------------------------------
            mesh_est = np.array(np.meshgrid(ind_real,ind_est))
            comb_est = mesh_est.T.reshape(-1, 2)
            #--------------------------------
            comb_real2 = comb_real[:,0] + comb_real[:,1]*1j
            comb_est2 = comb_est[:,0] + comb_est[:,1]*1j
            ind_remove = np.in1d(comb_est2,comb_real2)
            comb_est = comb_est[np.logical_not(ind_remove)]
            #--------------------------------
            CM[comb_est[:,0],comb_est[:,1]] += 1
        return CM, Temp
    
    for i in range(n_samples):
        CM,Temp = acum_CM(y_test_g[i,:],y_est_g[i,:],CM,Temp)
        
    return CM,Temp

## 1. Logistic regression

### 1.1. Entrenamiento

In [ ]:
clf = LogisticRegression(class_weight = 'balanced', max_iter=1000)
model_log = []
parameters = {'ML_model__penalty':['l2'], 'ML_model__C': [0.1, 0.5, 1]}
ML_mol(X_train, y_train, parameters, model_log, ML_model=clf)

### 1.2. Matriz de confusión

In [ ]:
prediction_lg, con_mat_lg = [], []
matrix_conf(model_log, X_test, y_test, prediction_lg, con_mat_lg)

### 1.3. Métricas globales

In [ ]:
print('Las métricas globales para el modelo de regresion logística son:')
global_metric(y_test, prediction_lg, con_mat_lg, metric=f1_score)
global_metric(y_test, prediction_lg, con_mat_lg, metric=accuracy_score)
global_metric(y_test, prediction_lg, con_mat_lg, metric=balanced_accuracy_score)
global_metric(y_test, prediction_lg, con_mat_lg, metric=recall_score)
global_metric(y_test, prediction_lg, con_mat_lg, metric=precision_score)

In [ ]:
print('Modelo de regresión logística:')
global_logloss(X_test, y_test, con_mat_lg, model_log)

### 1.4. Matriz de confusión global

In [ ]:
CM_lr, Temp = global_multiLabel_confusion_matrix(y_test, np.array(prediction_lg).T)

# #Normalized
suma = CM_lr.sum(axis=0) + Temp
for value, col in enumerate(suma.T):
    if col == 0:
        suma[value] = 1
    else:
        suma
CM_lr_no = CM_lr/suma

plt.figure(figsize=(30,30))
protein_names = np.arange(41)
protein = ytarget.columns
ax = sn.heatmap(CM_lr_no, annot=True, fmt='.1g', xticklabels = protein, yticklabels = protein_names)

plt.title('Global multi-label confusion matrix - Logistic regression')
plt.show()

## 2. Random Forest

### 2.1. Entrenamiento

In [ ]:
model_rf = RandomForestClassifier(criterion='entropy', class_weight='balanced', n_jobs=-1)
modelo_rf = []
parameters_rf = {'ML_model__n_estimators':[20,40,60,80,100,120],'ML_model__max_depth':[4,6,8,10,12]}
ML_mol(X_train, y_train, parameters_rf, modelo_rf, ML_model=model_rf)

### 2.2. Matriz de confusión

In [ ]:
prediction_rf, con_mat_rf = [], []
matrix_conf(modelo_rf, X_test, y_test, prediction_rf, con_mat_rf)

### 2.3. Métricas globales

In [ ]:
print('Las métricas globales para el modelo de arboles aleatorios son:')
global_metric(y_test, prediction_rf, con_mat_rf, metric=f1_score)
global_metric(y_test, prediction_rf, con_mat_rf, metric=accuracy_score)
global_metric(y_test, prediction_rf, con_mat_rf, metric=balanced_accuracy_score)
global_metric(y_test, prediction_rf, con_mat_rf, metric=recall_score)
global_metric(y_test, prediction_rf, con_mat_rf, metric=precision_score)

In [ ]:
print('Modelo Random Forest (Bosques aleatorios):')
global_logloss(X_test, y_test, con_mat_rf, modelo_rf)

### 2.4. Matriz de confusión global

In [ ]:
CM_rf, Temp = global_multiLabel_confusion_matrix(y_test, np.array(prediction_rf).T)

# #Normalized
suma = CM_rf.sum(axis=0) + Temp
for value, col in enumerate(suma.T):
    if col == 0:
        suma[value] = 1
    else:
        suma
CM_rf_no = CM_rf/suma

plt.figure(figsize=(30,30))
protein_names = np.arange(41)
protein = ytarget.columns
ax = sn.heatmap(CM_rf_no, annot=True, fmt='.1g', xticklabels = protein, yticklabels = protein_names)

plt.title('Global multi-label confusion matrix - Random Forest')
plt.show()

## 3. Support vector classifier (SVC)

### 3.1. Entrenamiento

In [ ]:
svm_c = SVC(class_weight = 'balanced', gamma = 'scale', probability=True)
SVM_model = []
parameters_svm = {'ML_model__kernel':['linear','rbf'], 'ML_model__C':[0.01, 0.1, 1, 10]}
ML_mol(X_train, y_train, parameters_svm, SVM_model, ML_model=svm_c)

### 3.2. Matriz de confusión

In [ ]:
prediction_svm, con_mat_svm = [], []
matrix_conf(SVM_model, X_test, y_test, prediction_svm, con_mat_svm)

### 3.3. Métricas globales

In [ ]:
print('Las métricas globales para el modelo de máquinas de soporte son:')
global_metric(y_test, prediction_svm, con_mat_svm, metric=f1_score)
global_metric(y_test, prediction_svm, con_mat_svm, metric=accuracy_score)
global_metric(y_test, prediction_svm, con_mat_svm, metric=balanced_accuracy_score)
global_metric(y_test, prediction_svm, con_mat_svm, metric=recall_score)
global_metric(y_test, prediction_svm, con_mat_svm, metric=precision_score)

### 3.4. Matriz de confusión global

In [ ]:
CM_svm, Temp = global_multiLabel_confusion_matrix(y_test, np.array(prediction_svm).T)

# #Normalized
suma = CM_svm.sum(axis=0) + Temp
for value, col in enumerate(suma.T):
    if col == 0:
        suma[value] = 1
    else:
        suma
CM_svm_no = CM_svm/suma

plt.figure(figsize=(30,30))
protein_names = np.arange(41)
protein = ytarget.columns
ax = sn.heatmap(CM_svm_no, annot=True, fmt='.1g', xticklabels = protein, yticklabels = protein_names)

plt.title('Global multi-label confusion matrix - Support Vector Classifier')
plt.show()

## 4. XGBoost classifier 

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder().fit(np.unique(y_train[:,0:1]))

In [ ]:
classes = []
for i in range(len(ytarget.columns)):
    class_we = compute_class_weight('balanced', le.classes_, y_train[:,i:i+1].flatten())
    classes.append(class_we)

In [ ]:
for val in classes:
    mean_1 = np.mean(val[1])
    mean_2 = np.mean(val[0])

### 4.1. Entrenamiento

In [ ]:
# parameters_gb = {'n_estimators':[20,40,60,80,100],'base_estimator__max_depth':[4,6,8,10]}
parameters_xgb = {'n_estimators':[40,60,80,100],'max_depth': [2,4,6,8], 'learning_rate':[1,0.5,0.1], 'reg_lambda':[0.2]}
estimator = XGBClassifier(objective='binary:logistic', scale_pos_weight=mean_1/mean_2, random_state=0,use_label_encoder=False, eval_metric='logloss')#, max_delta_step=3)
xgb_model = []
ML_mol(X_train, y_train.astype(int), parameters_xgb, estimator, xgb_model)

### 4.2. Matriz de confusión

In [ ]:
prediction_xgb, con_mat_xgb = [], []
matrix_conf(xgb_model, X_test, y_test, prediction_xgb, con_mat_xgb)

### 4.3. Métricas globales

In [ ]:
print('Las métricas globales para el modelo xgboost son:')
global_metric(y_test, prediction_xgb, con_mat_xgb, metric=f1_score)
global_metric(y_test, prediction_xgb, con_mat_xgb, metric=accuracy_score)
global_metric(y_test, prediction_xgb, con_mat_xgb, metric=balanced_accuracy_score)
global_metric(y_test, prediction_xgb, con_mat_xgb, metric=recall_score)
global_metric(y_test, prediction_xgb, con_mat_xgb, metric=precision_score)

In [ ]:
print('Modelo XGBoost:')
global_logloss(X_test, y_test, con_mat_xgb, xgb_model)

### 4.4. Matriz de confusión global

In [ ]:
CM_xgb, Temp = global_multiLabel_confusion_matrix(y_test, np.array(prediction_xgb).T)

# #Normalized
suma = CM_xgb.sum(axis=0) + Temp
for value, col in enumerate(suma.T):
    if col == 0:
        suma[value] = 1
    else:
        suma
CM_xgb_no = CM_xgb/suma

plt.figure(figsize=(30,30))
protein_names = np.arange(41)
protein = ytarget.columns
ax = sn.heatmap(CM_xgb_no, annot=True, fmt='.1g', xticklabels = protein, yticklabels = protein_names)

plt.title('Global multi-label confusion matrix - XGboost')
plt.show()